# Import libraries

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import researchpy as rp

from sklearn.model_selection import train_test_split
# calculate error
from sklearn.metrics import r2_score

import math
import time

## to lea: start here

## Normal equation

Normal equation is a method of using matrix multiplication to obtain the parameters of the best fit line.
<br><br>
Here, we will first define the normal equation functions.

In [2]:
def w_normaleqn(X,y):
    a = X.T.dot(X)
    b = X.T.dot(y)
    w = np.linalg.inv(a).dot(b)
    return w

def mseloss_normaleqn(X,w,y):
    n = len(X)
    Xwy = X.dot(w)-y
    loss= 1/n*Xwy.T.dot(Xwy)
    return loss

def f_pred(X,w):
    f = X.dot(w)
    return f 

In [3]:
# not sure if this is needed tbh
def grad_loss_normaleqn(y_pred,y,X):
    n = len(y)
    grad = 2/n*(X.T.dot(y_pred)-X.T.dot(y))
    return grad

### Preprocessing
We tried running training the normal equation model wih some different types of pre-processing on the dataset. The version shown below is the one that gives the best test accuracy (in terms of R2 score).

In [4]:
# import dataset
train = pd.read_csv('train.csv')

In [5]:
# Remove null
# For feature with too many missing values, remove this feature entirely.
# Set treshold to be 10% of all data points
# For feature with only a few missing values, remove that data point.

# from EDA
nullData = [['LotFrontage', 259], ['MasVnrArea', 8], ['Electrical', 1], ['GarageYrBlt', 81]]
n = len(train)
treshold = 0.1
drop = []

print('Drop feature - too many nulls:')
for i in nullData:
    if i[1]/n > treshold: # Arbitrary treshold: 10%
        print(i[0])
        train.drop(columns=[i[0]], inplace=True)
    else:
        drop.append(i[0])
        
print('Remove data point:')
print(drop)
train.dropna(subset=drop, inplace=True)

train.head()

Drop feature - too many nulls:
LotFrontage
Remove data point:
['MasVnrArea', 'Electrical', 'GarageYrBlt']


,Id,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Remove highly skewed categorical variables, with one category of data occuping >= 90% of data.

categorical = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond','Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

# skewness of categorical variables
max_percent = []
catogorical_skewed=[]
for i in categorical: 
    if rp.summary_cat(train[i])["Percent"].max() >= 90: 
        max_percent.append(rp.summary_cat(train[i])["Percent"].max())
print ("The number of variables with one category of data which occupies >= 90% of data =", len(max_percent))

# highly skewed categorical variables
for i in categorical: 
    if rp.summary_cat(train[i])["Percent"].max() >= 90:
        catogorical_skewed.append(i)
        print (i,"/ratio of the dominant category = ", rp.summary_cat(train[i])["Percent"].max()/100)

The number of variables with one category of data which occupies >= 90% of data = 15
Street /ratio of the dominant category =  0.9964
LandContour /ratio of the dominant category =  0.9015000000000001
Utilities /ratio of the dominant category =  0.9993000000000001
LandSlope /ratio of the dominant category =  0.9467
Condition2 /ratio of the dominant category =  0.9898
RoofMatl /ratio of the dominant category =  0.9818000000000001
BsmtCond /ratio of the dominant category =  0.9246
Heating /ratio of the dominant category =  0.981
CentralAir /ratio of the dominant category =  0.9495999999999999
Electrical /ratio of the dominant category =  0.9226000000000001
Functional /ratio of the dominant category =  0.9336
GarageQual /ratio of the dominant category =  0.9504
GarageCond /ratio of the dominant category =  0.9612999999999999
PavedDrive /ratio of the dominant category =  0.9372
MiscFeature /ratio of the dominant category =  0.9216


In [7]:
# Drop
for i in catogorical_skewed:
    train.drop(columns=[i], inplace=True)

In [8]:
for i in catogorical_skewed:
    categorical.remove(i)
#skewness of categorical variables
max_percent = []
for i in categorical: 
    if rp.summary_cat(train[i])["Percent"].max() >= 90: 
        max_percent.append(rp.summary_cat(train[i])["Percent"].max())
print ("The number of variables with one category of data which occupies >= 90% of data =", len(max_percent))
#highly skewed categorical variables
for i in categorical: 
    if rp.summary_cat(train[i])["Percent"].max() >= 90: 
         print (i,"/ratio of the dominant category = ", rp.summary_cat(train[i])["Percent"].max()/100)

The number of variables with one category of data which occupies >= 90% of data = 0


In [9]:
# One hot encoding
train = pd.get_dummies(train, columns= categorical, prefix= categorical)
train.head()

,Id,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,0,1,0,0,0,0,1,0
1,2,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,0,1,0,0,0,0,1,0
2,3,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,0,1,0,0,0,0,1,0
3,4,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,0,1,1,0,0,0,0,0
4,5,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,0,1,0,0,0,0,1,0


In [10]:
# Feature selection
# Choosing variables that is correlated to SalePrice with absolute value more than 0.2

# variables most closely related to SalePrice
corr = train.corr()['SalePrice']
# print(corr)
feature_select=[]
corrshape=corr.nlargest(corr.shape[0])[1:corr.shape[0]].shape[0]
# print(corrshape)
for i in range(corrshape-1):
    if (abs(corr.nlargest(corrshape)[1:corrshape][i])>0.2):
        feature_select.append(corr.nlargest(corrshape)[1:corrshape].axes[0][i])
        
print('Number of features selected = ', len(feature_select))        
print(corr.nlargest(corrshape)[1:corrshape])

Number of features selected =  68
GrLivArea            0.709783
GarageCars           0.636173
GarageArea           0.607197
TotalBsmtSF          0.603284
1stFlrSF             0.596087
                       ...   
OverallQual_5       -0.383080
GarageType_Detchd   -0.406550
BsmtQual_TA         -0.456964
GarageFinish_Unf    -0.485273
KitchenQual_TA      -0.527689
Name: SalePrice, Length: 246, dtype: float64


In [11]:
train = train[feature_select+['Id', 'SalePrice']]
train.head()

,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,BsmtQual_Ex,TotRmsAbvGrd,KitchenQual_Ex,YearBuilt,...,HeatingQC_TA,MasVnrType_None,Foundation_CBlock,OverallQual_5,GarageType_Detchd,BsmtQual_TA,GarageFinish_Unf,KitchenQual_TA,Id,SalePrice
0,1710,2,548,856,856,2,0,8,0,2003,...,0,0,0,0,0,0,0,0,1,208500
1,1262,2,460,1262,1262,2,0,6,0,1976,...,0,1,1,0,0,0,0,1,2,181500
2,1786,2,608,920,920,2,0,6,0,2001,...,0,0,0,0,0,0,0,0,3,223500
3,1717,3,642,756,961,1,0,7,0,1915,...,0,1,0,0,1,1,1,0,4,140000
4,2198,3,836,1145,1145,2,0,9,0,2000,...,0,0,0,0,0,0,0,0,5,250000


### Linear regression
Now, we will run the function on the pre-processed dataset.

In [12]:
X1 = train.copy()
X1.drop(['Id', 'SalePrice'], axis=1, inplace=True)
y1 = train['SalePrice'][:,None]

train_X1, test_X1, train_y1, test_y1 = train_test_split(X1,y1,test_size=0.2, shuffle=False)

train_X1.insert(0, 'ones', 1)
test_X1.insert(0, 'ones', 1)


start = time.time()
w1 = w_normaleqn(train_X1,train_y1)
print('Time =',time.time() - start)

y_pred = f_pred(train_X1, w1)
r2_train1 = r2_score(train_y1, y_pred)
print('Train R2 score \t=', r2_train1)

y_pred = f_pred(test_X1, w1)
r2_test1 = r2_score(test_y1, y_pred)
print('Test R2 score \t=', r2_test1)

Time = 0.018752574920654297
Train R2 score 	= 0.8827229771923898
Test R2 score 	= 0.6685651754863232
